### Učitavanje podataka

In [1]:
primjer = "Uvod u teorijsko računarstvo je zabavan!"

In [2]:
# Na razini riječi
print("Primjer 1 na razini riječi:    ", primjer.split(" "))

# Na razini slova
print("Primjer 2 na razini slova:     ", [p for p in primjer])

# Na razini "slogova"
print("Primjer 3 na razini \"slogova\": ", [primjer[i:i+3] for i in range(0, len(primjer), 3)])

Primjer 1 na razini riječi:     ['Uvod', 'u', 'teorijsko', 'računarstvo', 'je', 'zabavan!']
Primjer 2 na razini slova:      ['U', 'v', 'o', 'd', ' ', 'u', ' ', 't', 'e', 'o', 'r', 'i', 'j', 's', 'k', 'o', ' ', 'r', 'a', 'č', 'u', 'n', 'a', 'r', 's', 't', 'v', 'o', ' ', 'j', 'e', ' ', 'z', 'a', 'b', 'a', 'v', 'a', 'n', '!']
Primjer 3 na razini "slogova":  ['Uvo', 'd u', ' te', 'ori', 'jsk', 'o r', 'aču', 'nar', 'stv', 'o j', 'e z', 'aba', 'van', '!']


Tokenizator na razini riječi za veliku količinu teksta daje veliki vokabular -> Svaki oblik pojedine riječi zaseban je token unutar vokabulara; primjerice: "*o, doing, does, done*".

Tokenizator na razini znakova (slova) uvijek ima jednako velik vokabular, ali svaki pojedini token ne nosi semantičko značenje; primjerice "**a***lph***a***bet, st***a***in, ll***a***m***a**"

Tokenizator na razini podriječi ("slogova") se čini kao dobar odabir!


### Wordpiece

In [ ]:
# !conda install transformers

Započnimo s jednostavnim korpusom tekstova: 3 rečenice

In [1]:
corpus = [
    "Ovo su šeste vježbe iz Uvoda u teorijsko računarstvo.",
    "Na ovim vježbama raditi ćete tokenizaciju i jednostavnu analizu teksta.",
    "Ovo poglavlje raspravlja o WordPiece tokenizatoru.",
    "Dobrodošli!"
]

S obzirom da rekreiramo WordPiece tokenizator korišten u radu [BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding](https://arxiv.org/abs/1810.04805) iskoristiti ćemo pripremljenu predtokenizacijsku za navedeni model (BERT).

In [2]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

/home/p0l3/miniforge3/envs/utor/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


Izračunajmo frekvenciju svake pojedine riječi.

In [4]:
from collections import defaultdict

word_freqs = defaultdict(int)
for text in corpus:
    words_with_offsets = tokenizer.backend_tokenizer.pre_tokenizer.pre_tokenize_str(text)
    new_words = [word for word, offset in words_with_offsets]
    for word in new_words:
        word_freqs[word] += 1

print(word_freqs)

print(defaultdict(
    int, word_freqs))

defaultdict(<class 'int'>, {'Ovo': 2, 'su': 1, 'šeste': 1, 'vježbe': 1, 'iz': 1, 'Uvoda': 1, 'u': 1, 'teorijsko': 1, 'računarstvo': 1, '.': 3, 'Na': 1, 'ovim': 1, 'vježbama': 1, 'raditi': 1, 'ćete': 1, 'tokenizaciju': 1, 'i': 1, 'jednostavnu': 1, 'analizu': 1, 'teksta': 1, 'poglavlje': 1, 'raspravlja': 1, 'o': 1, 'WordPiece': 1, 'tokenizatoru': 1, 'Dobrodošli': 1, '!': 1})
defaultdict(<class 'int'>, {'Ovo': 2, 'su': 1, 'šeste': 1, 'vježbe': 1, 'iz': 1, 'Uvoda': 1, 'u': 1, 'teorijsko': 1, 'računarstvo': 1, '.': 3, 'Na': 1, 'ovim': 1, 'vježbama': 1, 'raditi': 1, 'ćete': 1, 'tokenizaciju': 1, 'i': 1, 'jednostavnu': 1, 'analizu': 1, 'teksta': 1, 'poglavlje': 1, 'raspravlja': 1, 'o': 1, 'WordPiece': 1, 'tokenizatoru': 1, 'Dobrodošli': 1, '!': 1})


Prisjetimo se abecede (alfabeta) koji je jedinstveni skup nastao od svih početnih slova riječi te svih ostalih slova u riječi s prefiksom "##".

In [5]:
alphabet = []
for word in word_freqs.keys():
    if word[0] not in alphabet:
        alphabet.append(word[0])
    for letter in word[1:]:
        if f"##{letter}" not in alphabet:
            alphabet.append(f"##{letter}")

alphabet.sort()
alphabet

print(alphabet)

['!', '##P', '##a', '##b', '##c', '##d', '##e', '##g', '##i', '##j', '##k', '##l', '##m', '##n', '##o', '##p', '##r', '##s', '##t', '##u', '##v', '##z', '##č', '##š', '##ž', '.', 'D', 'N', 'O', 'U', 'W', 'a', 'i', 'j', 'o', 'p', 'r', 's', 't', 'u', 'v', 'ć', 'š']


Također dodajemo posebne tokene u vokabular, u ovom slučaju `["[PAD]", "[UNK]", "[CLS]", "[SEP]", "[MASK]"]`:

In [6]:
vocab = ["[PAD]", "[UNK]", "[CLS]", "[SEP]", "[MASK]"] + alphabet.copy()

In [7]:
print(len(vocab))

48


Razdvojimo sve riječi s obzirom na početna slova iz vokabulara:

In [8]:
splits = {
    word: [c if i == 0 else f"##{c}" for i, c in enumerate(word)]
    for word in word_freqs.keys()
}
print(splits)

{'Ovo': ['O', '##v', '##o'], 'su': ['s', '##u'], 'šeste': ['š', '##e', '##s', '##t', '##e'], 'vježbe': ['v', '##j', '##e', '##ž', '##b', '##e'], 'iz': ['i', '##z'], 'Uvoda': ['U', '##v', '##o', '##d', '##a'], 'u': ['u'], 'teorijsko': ['t', '##e', '##o', '##r', '##i', '##j', '##s', '##k', '##o'], 'računarstvo': ['r', '##a', '##č', '##u', '##n', '##a', '##r', '##s', '##t', '##v', '##o'], '.': ['.'], 'Na': ['N', '##a'], 'ovim': ['o', '##v', '##i', '##m'], 'vježbama': ['v', '##j', '##e', '##ž', '##b', '##a', '##m', '##a'], 'raditi': ['r', '##a', '##d', '##i', '##t', '##i'], 'ćete': ['ć', '##e', '##t', '##e'], 'tokenizaciju': ['t', '##o', '##k', '##e', '##n', '##i', '##z', '##a', '##c', '##i', '##j', '##u'], 'i': ['i'], 'jednostavnu': ['j', '##e', '##d', '##n', '##o', '##s', '##t', '##a', '##v', '##n', '##u'], 'analizu': ['a', '##n', '##a', '##l', '##i', '##z', '##u'], 'teksta': ['t', '##e', '##k', '##s', '##t', '##a'], 'poglavlje': ['p', '##o', '##g', '##l', '##a', '##v', '##l', '##j', '##

Napišimo funkciju koja izračunava "score" za svaki par tokena.

In [9]:
def compute_pair_scores(splits):
    
    letter_freqs = defaultdict(int) # Riječnik koji prati koliko se pojedini element puta pojavi u tekstu
    pair_freqs = defaultdict(int) # Prati koliko se učestalo pojedini par pojavljuje u tekstu
    
    for word, freq in word_freqs.items():
        split = splits[word]
        if len(split) == 1:
            letter_freqs[split[0]] += freq
            continue
        for i in range(len(split) - 1):
            pair = (split[i], split[i + 1])
            letter_freqs[split[i]] += freq
            pair_freqs[pair] += freq
        letter_freqs[split[-1]] += freq

    scores = {
        pair: freq / (letter_freqs[pair[0]] * letter_freqs[pair[1]])
        for pair, freq in pair_freqs.items()
    }
    return scores

Pogledajmo "score" prvih 5 parova nakon inicijalnog razdvajanja riječi:

In [10]:
pair_scores = compute_pair_scores(splits)
for i, key in enumerate(pair_scores.keys()):
    print(f"{key}: {pair_scores[key]}")
    if i >= 5:
        break

('O', '##v'): 0.125
('##v', '##o'): 0.03333333333333333
('s', '##u'): 0.16666666666666666
('š', '##e'): 0.06666666666666667
('##e', '##s'): 0.011111111111111112
('##s', '##t'): 0.09523809523809523


Pronađimo par s najboljim "score":

In [11]:
best_pair = ""
max_score = None
for pair, score in pair_scores.items():
    if max_score is None or max_score < score:
        best_pair = pair
        max_score = score

print(best_pair, max_score)

('##ž', '##b') 0.3333333333333333


Dakle prvo spajanje koje učimo je `("##ž", "##b") -> "##žb"`.

In [12]:
vocab.append("##žb")

Definirajmo funkciju za spajanje parova:

In [13]:
def merge_pair(a, b, splits):
    for word in word_freqs:
        split = splits[word]
        if len(split) == 1:
            continue
        i = 0
        while i < len(split) - 1:
            if split[i] == a and split[i + 1] == b:
                merge = a + b[2:] if b.startswith("##") else a + b
                split = split[:i] + [merge] + split[i + 2 :]
            else:
                i += 1
        splits[word] = split
    return splits

Pogledajmo rezultat prvog spajanja:

In [14]:
splits = merge_pair("##ž", "##b", splits)
splits["vježbe"]

['v', '##j', '##e', '##žb', '##e']

Kreirajmo vokabular sa 100 tokena:

In [15]:
vocab_size = 100
while len(vocab) < vocab_size:
    scores = compute_pair_scores(splits)
    best_pair, max_score = "", None
    for pair, score in scores.items():
        if max_score is None or max_score < score:
            best_pair = pair
            max_score = score
    splits = merge_pair(*best_pair, splits)
    new_token = (
        best_pair[0] + best_pair[1][2:]
        if best_pair[1].startswith("##")
        else best_pair[0] + best_pair[1]
    )
    vocab.append(new_token)

In [22]:
print(vocab)

['[PAD]', '[UNK]', '[CLS]', '[SEP]', '[MASK]', '!', '##P', '##a', '##b', '##c', '##d', '##e', '##g', '##i', '##j', '##k', '##l', '##m', '##n', '##o', '##p', '##r', '##s', '##t', '##u', '##v', '##z', '##č', '##š', '##ž', '.', 'D', 'N', 'O', 'U', 'W', 'a', 'i', 'j', 'o', 'p', 'r', 's', 't', 'u', 'v', 'ć', 'š', '##žb', '##gl', '##šl', '##dP', 'su', '##ču', 'vj', '##čun', 'an', '##lj', '##sp', '##spr', '##rdP', '##br', 'Ov', 'Uv', '##vlj', 'ov', 'iz', '##ju', '##js', '##jsk', '##st', '##stv', '##rstv', '##vn', '##vnu', '##ru', '##zu', '##kst', '##ri', '##rijsk', 'ovi', 'ovim', '##iz', '##niz', '##dn', '##iju', '##ciju', '##li', '##lizu', '##rdPi', '##šli', '##di', '##dit', '##diti', 'Ovo', 'Uvo', 'Uvod', '##orijsk', '##orijsko', '##rstvo']


In [17]:
def encode_word(word):
    tokens = []
    while len(word) > 0:
        i = len(word)
        while i > 0 and word[:i] not in vocab:
            i -= 1
        if i == 0:
            return ["[UNK]"]
        tokens.append(word[:i])
        word = word[i:]
        if len(word) > 0:
            word = f"##{word}"
    return tokens

In [18]:
print(encode_word("vježba"))
print(encode_word("HOgging"))

['vj', '##e', '##žb', '##a']
['[UNK]']


In [19]:
def tokenize(text):
    pre_tokenize_result = tokenizer._tokenizer.pre_tokenizer.pre_tokenize_str(text)
    pre_tokenized_text = [word for word, offset in pre_tokenize_result]
    encoded_words = [encode_word(word) for word in pre_tokenized_text]
    return sum(encoded_words, [])

In [20]:
print(tokenize("Ovo su vježbe iz Uvoda u teorijsko računarstvo"))

['Ovo', 'su', 'vj', '##e', '##žb', '##e', 'iz', 'Uvod', '##a', 'u', 't', '##e', '##orijsko', 'r', '##a', '##čun', '##a', '##rstvo']


In [21]:
print(len(vocab))

100
